In [1]:
import os
from os import listdir, makedirs
from os.path import join, exists, expanduser

from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.applications.resnet50 import ResNet50, preprocess_input
import tensorflow as tf

In [2]:
img_width,img_height = 299,299


In [3]:
%pwd

'F:\\workspace\\jupyterWorkSpace'

In [4]:
train_data_dir = 'F:/workspace/jupyterWorkSpace/archive/fruits-360/Training/'
validation_data_dir = 'F:/workspace/jupyterWorkSpace/archive/fruits-360/Test/'

In [5]:
batch_size = 16
nb_train_sapmles = 67692
nb_validatin_samples = 3063

In [6]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)

test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height,img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height,img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 67692 images belonging to 131 classes.
Found 22688 images belonging to 131 classes.


In [7]:
weight_dir_notop = 'F:/workspace/jupyterWorkSpace/archive/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5'

In [8]:

base_model = ResNet50(weights = 'imagenet',include_top=False)

In [9]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512,activation='relu')(x)
predictions = Dense(131,activation='softmax')(x)
model = Model(inputs=base_model.input,outputs=predictions)

In [10]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=optimizers.SGD(lr=1e-4,momentum=0.9),
    metrics=['accuracy']
)
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, None, None, 6 256         conv1_conv[0][0]                 
_______________________________________________________________________________________

In [ ]:
with tf.device('/GPU:0'):
    history_pretrained = model.fit_generator(
        train_generator,
        epochs=10,
        shuffle=True,
        verbose = 1,
        validation_data = validation_generator
    )

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
   3/4231 [..............................] - ETA: 7:17:17 - loss: 5.1205 - accuracy: 0.0208

In [ ]:
xception_transfer.save('my_model')

In [ ]:
xception_transfer.save('xception_TL')